In [1]:
import pandas as pd
import numpy as np
import re

# Pre-processamento
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

# Classificadores
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression

from warnings import filterwarnings

# Ignorando warnings de deprecation
filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [3]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [4]:
data.shape

(2606, 2)

In [5]:
# Removendo o atributo ID (irrelevante)
ids = data['ID']
data.drop(['ID'], axis=1, inplace=True)

In [6]:
# Inicializando atributos target com 0.0
data['SMARTPHONE'] = np.zeros((len(data)))

# Parte 1: Anotação dos dados
Objetivando a aplicação de um algoritmo supervisionado, foram classificados manualmente os dados entre 1.0 (Smartphone) e 0.0 (Não-Smartphone) e depois filtrados os possíveis casos indesejáveis. Tal tarefa é relativamente simples, pois os intervalos onde há smartphones são facilmente identificáveis. Caso os dados estivessem muito embaralhados e/ou o conjunto de dados fosse muito grande, outras técnicas poderiam ser usadas (i.e. clustering, regex).

In [7]:
# Manualmente selecionando os intervalos onde existem smartphones
data['SMARTPHONE'].iloc[6:39] = 1.0
data['SMARTPHONE'].iloc[2035:] = 1.0

# Nota: Há alguns smartwatches nesse intervalo.
#       Além disso, pode ser que haja algum smartphone que não esteja nesse intervalo.
#       Esses casos serão tratados a seguir

In [8]:
# Casos de Smartwatches classificados como 1.0
for i, example in data[data['SMARTPHONE'] == 1.0]['TITLE'].iteritems():
    if re.findall('watch|rel[ó|o]gio', example.lower()):
        print(i, example)
        data['SMARTPHONE'].iloc[i] = 0.0

2207 Apple Watch Série 3 42mm Cores E Edição  Nike+ Em Estoque
2227 Apple Watch Series 3 42mm Nike + Gps Prova D'água Lacrado
2244 Apple Watch Series 3 42mm Gps Garantia Apple + Brinde
2253 Relógio Samsung Gear S3 Sm-r770 Classic Wifi Bluetooth
2288 Apple Watch Series 3  42mm Nike + Gps Prova D'água Lançameto
2289 Apple Watch Series 3 42mm Nike +  Gps Prova D'água
2298 Apple Watch Series 3 38mm Nike + Gps Prova D'água
2301 Apple Watch Nike+ Séries 3 42mm Lacrado
2302 Apple Watch Nike+ Séries 3 38mm Lacrado Novo
2303 Apple Watch Nike+ Séries 3 38mm Lacrado
2309 Apple Watch Series 3 38mm Nike + Gps Prova D'água Lacrado
2314 Apple Watch Série 3 42mm Gps + 2 Pulseiras Extras
2333 Relógio Samsung S3 Frontier + 2 Pulseiras Extras
2345 Apple Watch Series 3 42mm Lacrado + Brinde
2353 Apple Watch Nike+ Séries 3 42mm Lacrado + Envio Imediato
2380 Apple Watch Series 3 38mm Lacrado Nf + Brinde
2417 Smartwatch Samsung Gear S3 Frontier 1.3 Bluetooth - Original
2597 Relógio Smartwatch Samsung Gear Sm

In [9]:
# Casos de smartphones classificados como 0.0
for i, example in data[data['SMARTPHONE'] == 0.0]['TITLE'].iteritems():
    if re.findall('^smartphone|^celular', example.lower()):
        print(i, example)
        
# Somente 2 casos
data['SMARTPHONE'].iloc[1892:1894] = 1.0

# Nota: Foram encontrados alguns celulares, mas que não são smartphones. Estes foram desconsiderados.

548 Celular Multilaser Up- Branco/Rosa
775 Celular Blu Samba Jr Q53 Prata Dual Chip Silver
1886 Celular Up 2chip Quad Cam Mp3/4 Fm Branco/Rosa
1892 Smartphone Samsung Galaxy J2 Prime TV Dourado com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz
1893 Smartphone Samsung Galaxy J2 Prime TV Preto com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz


In [10]:
data['SMARTPHONE'].value_counts()

0.0    2018
1.0     588
Name: SMARTPHONE, dtype: int64

In [11]:
data['SMARTPHONE'].mean()
# Classes estão desbalanceadas

0.2256331542594014

# Parte 2: Pré-processamento
Nessa etapa serão feitos processos para normalizar os dados. Serão removidas stopwords, todas as palavras serão transformadas em representações de letras minúsculas e então será feita tokenização e bag of words.

In [12]:
# Adicionando pontuações às stopwords em português
pontuacoes = ['-', '.', ',', '(', ')']
pt_stopwords = stopwords.words('portuguese')

for pontuacao in pontuacoes:
    pt_stopwords.append(pontuacao)

In [13]:
# CountVectorizer remove os stopwords, tokeniza e obtem bag of words
vect = CountVectorizer(stop_words=pt_stopwords)
vect = CountVectorizer().fit(data['TITLE'])

data_vect = vect.transform(data['TITLE'])

In [14]:
data_vect

<2606x4165 sparse matrix of type '<class 'numpy.int64'>'
	with 21947 stored elements in Compressed Sparse Row format>

# Parte 3: Seleção do Modelo e Validação
Será utilizado Grid Search para encontrar os melhores parâmetros para cada classificador. O GS tem 5 folds de cross validation, método de avaliação f1 (pois as classes estão desbalanceadas) e é feita paralelização em 4 threads. Os classificadores escolhidos para a tarefa foram:

### SVM:
- Funciona bem em dados com alta dimensão e poucos exemplos (dataset está com 2606 linhas e 4165 colunas);
- Variando parâmetros C e gamma.

### Regressão Logística:
- Modelo simples e eficiente;
- Variando penalidades l1 e l2 e o parâmetro C.

### Bernoulli:
- Naive Bayes de representação binária;
- Variando constante de suavização de laplace.

### Multinomial:
- Naive Bayes para representação de múltiplas ocorrências;
- Variando constante de suavização de laplace.

In [15]:
# Definindo os parâmetros a serem selecionados para cada classificador
SVM = {'name': 'svm',
       'classifier': SVC(),
       'parameters': {'kernel': ['linear'],
       'C': [0.1, 1.0, 5, 10, 15],
       'gamma': [0.01, 0.1, 1.0, 5.0, 10],}}

Logistic = {'name': 'logistic_regression',
       'classifier': LogisticRegression(),
       'parameters': {'penalty': ['l1', 'l2'],
                      'C': [0.01, 0.1, 1.0, 5.0, 10],
                      'solver': ['liblinear']}}

Bernoulli = {'name': 'bernoulli_naive_bayes',
             'classifier': BernoulliNB(),
             'parameters': {'alpha': [1.0, 2.0, 5.0]}}
            
Multinomial = {'name': 'multinomial_naive_bayes',
               'classifier': MultinomialNB(),
               'parameters': {'alpha': [1.0, 2.0, 5.0]}}

classificadores = [SVM, Logistic, Bernoulli, Multinomial]

In [16]:
# Encontra os melhores parâmetros para cada classificador

modelos = {}
for classificador in classificadores:
    gs = GridSearchCV(estimator=classificador['classifier'],
                                param_grid=classificador['parameters'],
                                scoring='f1',
                                verbose=0,
                                cv=5,
                                n_jobs=4)

    gs.fit(data_vect, data['SMARTPHONE'])
    modelos[classificador['name']] = gs.best_estimator_

In [17]:
# Avaliação usando cross validation score
resultados = {}
for nome, modelo in modelos.items():
    resultados[nome] = np.mean(cross_val_score(modelo,
                                            data_vect,
                                            data['SMARTPHONE'],
                                            cv=5,
                                            scoring='f1'))
    
for nome, resultado in resultados.items():
    print(nome+':', resultado)
    
# Regressor Logístico > SVM > BernoulliNB > MultinomialNB

svm: 0.9820169917387778
logistic_regression: 0.9835542996094679
bernoulli_naive_bayes: 0.9485674493873768
multinomial_naive_bayes: 0.9203615982819263


In [18]:
# 100 principais features associadas a smartphones
feature_names = np.array(vect.get_feature_names())
coefs_sorted = modelos['logistic_regression'].coef_[0].argsort()
feature_names[coefs_sorted][-100:]

array(['1sim', '6gb', 'mini', 'xt1676', 'j5', 'g6', 'indigo', 'j810m',
       'orig', '20mp', 'câmera', 'g570m', 'j8', 'g5', 'duro', 'termica',
       'queda', 'na', 'xperia', 'pronta', 'leitor', 'grátis', '3gb',
       'entrega', 'no', 'grafite', 'pocket', 'a1', '2017', 'digital',
       'test', 'nuevo', 'ofertar', 'item', 'sim', 'frete', 'processador',
       '32', 's5', 'duos', 'promoção', '4gb', 'ano', 'android', 'tv',
       'j400m', 'j4', 's60', 'caterpillar', 'cat', 'k10', 'play', 'redmi',
       'e5', 'global', 'chip', 'apple', 'samsung', 'desbloqueado',
       '128gb', 'lg', 'xt1726', 'mi', 'fiscal', 'ram', '8mp', 'nf',
       'garantia', 'prime', 'nota', '8gb', 'moto', '3g', 'gb', 'j7',
       'ferrari', 'anatel', 'novo', 'vitrine', 'dual', 'nextel',
       'dourado', '4g', 'lacrado', 'g800h', '16gb', 'tela', 'plus',
       'original', 'xiaomi', '256gb', 'celular', 's9', 'galaxy',
       'motorola', 'smartphone', 'i867', 'iphone', '32gb', '64gb'],
      dtype='<U23')

In [19]:
# 100 principais features associadas a não-smartphones
feature_names[coefs_sorted][:100]

array(['capa', 'bumper', 'bracadeira', 'para', 'película', 'vidro', 'tab',
       'rosa', 'kit', 'watch', 'tablet', 'tpu', 'p355m', 'azul',
       'transparente', 'vermelho', 'pc', 'de', 'bumber', 's3', '5mp',
       'fonte', 'series', '38mm', 'pelicula', 'em', 'protetora',
       'shampoo', 'preta', 'único', 'capapelicula', 'pingente', '4s',
       'ero', 'laço', 'cabo', 'papel', 'do', 'silver', 'nike', 's6102',
       's4', 'samba', 'jr', 'q53', 'carregador', 'flip', 'ml', 'blu',
       'cm', 'ciabrink', 'premium', 'dados', 'couro', 'séries', '42mm',
       'core', 'pulseira', 'amarelo', 'ghz', 'violão', 'sm', 'infantil',
       'temperado', 'litocart', 'branco', 'mirror', 'usb', 'black',
       'adesivo', 'madeira', 'espelhamento', 'au906', 'cast', 'esportiva',
       'batom', 'verde', 'micro', 'vgn', 'veicular', 'adaptador',
       'braçadeira', 'idea', 'gps', '5c', 'tracta', 'whey', 'exclusiva',
       'personalizada', 'branca', 'ddr3', 'com', 'suporte', 'coração',
       'botão',

# Parte 4: Classificação
Aplicação do melhor modelo encontrado e salvamento do resultado.

In [20]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [21]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [22]:
vect = CountVectorizer(stop_words=pt_stopwords)
X = CountVectorizer().fit_transform(data['TITLE'])
y = modelos['logistic_regression'].predict(X)

In [23]:
modelos['logistic_regression']

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
data['SMARTPHONE'] = y

# SEGUNDA FASE

In [25]:
import nltk
from fuzzywuzzy import process

In [26]:
birdie_data = pd.read_csv('data/resultados.tsv', sep='\t')
submarino_data = pd.read_csv('data/submarino_smartphones.csv')

In [27]:
submarino_data.head()

,url,smartphone
0,https://www.submarino.com.br/categoria/celular...,Smartphone Motorola One Vision 128GB Dual Chip...
1,https://www.submarino.com.br/categoria/celular...,"Iphone 8 Plus 64GB Ouro Tela 5.5"" iOS 12 4G C..."
2,https://www.submarino.com.br/categoria/celular...,iPhone 8 Plus 64gb Silver Tela 5.5” iOS 12 4G ...
3,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy S10e 128GB Dual Chip...
4,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy Note 9 128GB Nano Ch...


In [28]:
vect = CountVectorizer().fit(data['TITLE'])
X = vect.transform(submarino_data['smartphone'])
y = modelos['multinomial_naive_bayes'].predict(X)
submarino_data['SMARTPHONE'] = y

In [29]:
submarino_data['SMARTPHONE'].value_counts()

1.0    4118
0.0     562
Name: SMARTPHONE, dtype: int64

In [30]:
submarino_data[submarino_data['SMARTPHONE'] == 0]

,url,smartphone,SMARTPHONE
167,https://www.submarino.com.br/categoria/celular...,Smartwatch Samsung Galaxy Watch Active Sm-r500...,0.0
197,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy Note 9 Preto Muito Bom -...,0.0
246,https://www.submarino.com.br/categoria/celular...,Seminovo: Samsung Galaxy J7 Prime Rosa Usado,0.0
321,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy A7 2016 Rosa Muito Bom -...,0.0
349,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy J5 Prime Preto,0.0
350,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy J5 Prime Rosa,0.0
352,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy J5 2016 Metal Preto Muit...,0.0
356,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy A5 2016 Preto Muito Bom ...,0.0
360,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy A3 Duos Branco,0.0
372,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy J1 Mini Preto Muito Bom ...,0.0


In [31]:
submarino_data = pd.DataFrame(submarino_data['smartphone'][submarino_data['SMARTPHONE'] == 1])
submarino_data.columns = ['TITLE']

birdie_data = pd.DataFrame(birdie_data['TITLE'][birdie_data['SMARTPHONE'] == 1], columns=['TITLE'])

In [32]:
print('tamanho (submarino):', len(submarino_data['TITLE']))
print('qtd de smartphones unicos:', submarino_data['TITLE'].nunique())

tamanho (submarino): 4118
qtd de smartphones unicos: 3376


In [33]:
print('tamanho (birdie):', len(birdie_data['TITLE']))
print('qtd de smartphones unicos:', birdie_data['TITLE'].nunique())

tamanho (birdie): 588
qtd de smartphones unicos: 546


In [34]:
submarino_data['TITLE'] = submarino_data['TITLE'].apply(lambda x: x.lower())

In [35]:
submarino_data.head()

,TITLE
0,smartphone motorola one vision 128gb dual chip...
1,"iphone 8 plus 64gb ouro tela 5.5"" ios 12 4g c..."
2,iphone 8 plus 64gb silver tela 5.5” ios 12 4g ...
3,smartphone samsung galaxy s10e 128gb dual chip...
4,smartphone samsung galaxy note 9 128gb nano ch...


In [36]:
birdie_data['TITLE'] = birdie_data['TITLE'].apply(lambda x: x.lower())

In [37]:
birdie_data.head()

,TITLE
6,smartphone lg optimus l7 ii dual p716 preto co...
7,smartphone samsung galaxy s4 mini duos preto c...
8,smartphone samsung galaxy s5 sm-g900m branco ...
9,smartphone samsung galaxy s5 sm-g900m dourado...
10,smartphone lg l90 dual d410 preto com tela de ...


In [38]:
re_marcas = ('samsung|lg|motorola|asus|apple|multilaser|sony|xiaomi|' + 
             'alcatel|positivo|microsoft|lenovo|huawei|caterpillar')

re_qtds_memoria = '[ ,\.]\d{1}gb[ ,\.]'

re_qtds_armazenamento = '[ ,\.]\d{2,3}gb[ ,\.]'

re_cor = 'azul|amarelo|vermelho|branco|preto|dourado|prata|violeta|grafite|black piano'

# re_dual_chip = 'dual chip|dual sim'

re_tam_tela = '[4-7][\.,]\d'

re_modelos = ('galaxy \w\d{1,2} prime|galaxy \w\d{1,2} mini|galaxy \w\d{1,2} edge|galaxy \w\d{1,2}\w?\+?|galaxy .* duos|galaxy note \d|' + # Samsung
              'moto \w\d? plus|moto \w\d?|' + # Motorola
              'redmi note \d\w? lite|redmi note \d\w?|mi \w?\d|mi \w?\d lite|pocophone \w\d{1,2}|' + # Xiaomi
              'iphone [\dx] plus|iphone [\dx]|' + # Apple
              'optimus \w\d{1,2}|\w\d{1,2} lite|k\d{1,2}|' + # LG
              'zenfone \d|' + # Asus
              'cat \w\d{1,2}|' + # Caterpillar
              'xperia \w{1,2}\d plus \w*\d{1,4}|xperia \w{1,2}\d \w*\d{1,4}|' + # Sony
              'vibe k\d \w*|k\d \w*')

res = {'MARCA': re_marcas,
       'MODELO': re_modelos,
       'QTD_MEMORIA': re_qtds_memoria,
       'QTD_ARMAZENAMENTO': re_qtds_armazenamento,
       }

In [39]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in birdie_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            if encontrado[0][-1] in [',', '.']: # conserta regex de qtd memoria/armazenamento
                encontrados.append(encontrado[0][:-1].strip())
            else:
                encontrados.append(encontrado[0].strip())
        else: 
            encontrados.append('')
        i += 1
    
    birdie_data[atributo] = encontrados
    birdie_data.reset_index(drop=True,inplace=True)

In [40]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in submarino_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            if encontrado[0][-1] in [',', '.', ' ']: # conserta regex de qtd memoria/armazenamento
                encontrados.append(encontrado[0][:-1])
            else:
                encontrados.append(encontrado[0])
        else: 
            encontrados.append('')
        i += 1
    
    submarino_data[atributo] = encontrados
    submarino_data.reset_index(drop=True,inplace=True)

In [41]:
birdie_data

,TITLE,MARCA,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 preto co...,lg,optimus l7,4gb,
1,smartphone samsung galaxy s4 mini duos preto c...,samsung,galaxy s4 mini,,
2,smartphone samsung galaxy s5 sm-g900m branco ...,samsung,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m dourado...,samsung,galaxy s5,,
4,smartphone lg l90 dual d410 preto com tela de ...,lg,,,
5,smartphone samsung galaxy core plus branco co...,samsung,,,
6,smartphone samsung galaxy pocket 2 duos sm-g11...,samsung,galaxy pocket 2 duos,,
7,smartphone samsung galaxy s5 duos sm-g900 pre...,samsung,galaxy s5,,
8,smartphone samsung galaxy ace 4 lite duos g313...,samsung,galaxy ace 4 lite duos,,
9,smartphone lg g2 lite d295 branco com tela de ...,lg,g2 lite,,


In [42]:
birdie_data['MARCA'].value_counts()

samsung        150
motorola       125
xiaomi          96
apple           67
                57
lg              32
asus            27
caterpillar      9
sony             8
lenovo           5
alcatel          4
multilaser       3
huawei           2
positivo         2
microsoft        1
Name: MARCA, dtype: int64

In [43]:
birdie_data['MODELO'].value_counts().sum()

588

In [44]:
birdie_data.head()

,TITLE,MARCA,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 preto co...,lg,optimus l7,4gb,
1,smartphone samsung galaxy s4 mini duos preto c...,samsung,galaxy s4 mini,,
2,smartphone samsung galaxy s5 sm-g900m branco ...,samsung,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m dourado...,samsung,galaxy s5,,
4,smartphone lg l90 dual d410 preto com tela de ...,lg,,,


In [45]:
birdie_smartphones = []
smartphone = ''
for i in range(len(birdie_data)):
    for attribute in birdie_data.columns[1:]:
        spec = str(birdie_data[attribute][i])
        if spec == '':
            pass
        else:
            smartphone += spec + ' '
        
    smartphone = smartphone.strip()
    birdie_smartphones.append(smartphone)
    smartphone = ''

In [46]:
submarino_smartphones = []
smartphone = ''
for i in range(len(submarino_data)):
    for attribute in submarino_data.columns[1:]:
        spec = str(submarino_data[attribute][i])
        if spec == '':
            pass
        else:
            smartphone += spec + ' '
        
    smartphone = smartphone.strip()
    submarino_smartphones.append(smartphone)
    smartphone = ''

In [47]:
matches = []
i = 0
for smartphone in birdie_smartphones:
    match = process.extractOne(smartphone, submarino_smartphones)
    matches.append(match)
    if i == 20:
        break
    i += 1

In [48]:
indices = []
for match in matches:
    indices.append(submarino_smartphones.index(match[0]))

In [49]:
df = pd.DataFrame(columns=['birdie','submarino'])
df['birdie'] = birdie_data.iloc[:, 0]
df['submarino'] = submarino_data.iloc[indices, 0].reset_index(drop=True)

In [50]:
df

,birdie,submarino
0,smartphone lg optimus l7 ii dual p716 preto co...,smartphone lg optimus l7 ii desbloqueado tim a...
1,smartphone samsung galaxy s4 mini duos preto c...,smartphone samsung galaxy s4 mini duos dual ch...
2,smartphone samsung galaxy s5 sm-g900m branco ...,smartphone samsung galaxy s5 - preto
3,smartphone samsung galaxy s5 sm-g900m dourado...,smartphone samsung galaxy s5 - preto
4,smartphone lg l90 dual d410 preto com tela de ...,smartphone lg optimus g branco android 4.1 des...
5,smartphone samsung galaxy core plus branco co...,smartwatch samsung galaxy watch active sm-r500...
6,smartphone samsung galaxy pocket 2 duos sm-g11...,smartphone samsung galaxy pocket duos s5302 br...
7,smartphone samsung galaxy s5 duos sm-g900 pre...,smartphone samsung galaxy s5 - preto
8,smartphone samsung galaxy ace 4 lite duos g313...,smartwatch samsung galaxy watch active sm-r500...
9,smartphone lg g2 lite d295 branco com tela de ...,smartphone lg g2 lite d295 dual preto/ titânio
